# Using GridSearchCV with Tensorflor: Classification

In [ ]:
import tensorflow as tf
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
# Load data
dataObj = load_breast_cancer()
X = dataObj.data
y = dataObj.target

# Splitting data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.30, random_state=1
)

# Standardization
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [ ]:
def get_clf_model(hidden_layer_sizes, meta, compile_kwargs):

    model = tf.keras.Sequential()
    input_layer = tf.keras.layers.Input(shape=(meta["n_features_in_"]))
    model.add(input_layer)

    for i, hidden_layer_size in enumerate(hidden_layer_sizes):
        hidden_layer = tf.keras.layers.Dense(
            units=hidden_layer_size, activation="relu", name=f"hidden{i+1}"
        )
        model.add(hidden_layer)

    if meta["target_type_"] == "binary":
        n_output_units = 1
        output_activation = "sigmoid"
        loss = "binary_crossentropy"
    elif meta["target_type_"] == "multiclass":
        n_output_units = meta["n_classes_"]
        output_activation = "softmax"
        loss = "sparse_categorical_crossentropy"
    else:
        raise NotImplementedError(f"Unsupported task type: {meta['target_type_']}")

    output_layer = tf.keras.layers.Dense(n_output_units, activation=output_activation)
    model.add(output_layer)

    model.compile(
        optimizer=compile_kwargs["optimizer"],
        loss=loss,
        metrics=["accuracy"],
    )

    return model

In [ ]:
# Note that I am setting validation_split=0, because already use GridSearchCV which has cross-validation.

clf = KerasClassifier(
    model=get_clf_model,
    hidden_layer_sizes=(16, 3),
    optimizer="adam",
    optimizer__learning_rate=0.001,
    batch_size=16,
    epochs=200,
    validation_split=0,
    verbose=1,
)

In [ ]:
set1 = {
    "optimizer__learning_rate": [0.01, 0.1],
    "hidden_layer_sizes": [(16, 3), (8, 3)],
}

param_grid = [set1]

In [ ]:
gs = GridSearchCV(estimator=clf, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  cv=5,
                  n_jobs=-1)

In [ ]:
gs.fit(X_train_std,y_train)

In [ ]:
df = pd.DataFrame(gs.cv_results_)
df = df.sort_values(by=['rank_test_score'])
display(df.head())

In [ ]:
gs.best_params_

In [ ]:
gs.predict(X_test_std)
testing_accuracy = gs.score(X_test_std,y_test)
print(f"Testing accuracy: {testing_accuracy:6.3f}")